# 本セクションの目次
1. IoT/Kafka/Spark Streamingの組み合わせ概要説明
2. Web画面からのデータ送信
3. ウィンドウ処理
4. DIKWモデル
5. UUIDの付与処理とイベント時間の付与

# IoT/Kafka/Spark Streamingを組み合わせてみよう

今回は、IoTとしてWebブラウザを利用してユーザのウェブ画面上での操作をトラッキングしてみたいと思います。

Web画面には3つの機能があります
1. ログイン(/でアクセスするとログインしたということにします)　http://nodejs:3000
2. カートに入れる(/cart)　http://nodejs:3000/cart
3. 決済する(/done) http://nodejs:3000/done

ユーザがこれらのアクションを取ったらkafkaへデータが飛んでくるような仕組みになっています。

実際の現場では、これらのデータをもとに分析を行い対策を練っていきます。  
ex. カートにまで入るけど決済されないなぁ



In [1]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [2]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,org.apache.spark:spark-avro_2.12:3.2.1") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2.1を使っています。)。

:: loading settings :: url = jar:file:/home/pyspark/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pyspark/.ivy2/cache
The jars for the packages stored in: /home/pyspark/.ivy2/jars
org.apache.spark#spark-streaming_2.13 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d5c338c8-21d5-4030-90a9-418a9d6c4783;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in ce

In [24]:
# kafkaからデータを読み取る設定を行います。
# 今回はpyspark-topic3から読み取ります

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic3") \
  .load()

In [4]:
# メモリシンク
memory_sink = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("memory") \
  .queryName("web_actions") \
  .trigger(processingTime="5 seconds") \
  .option("checkpointLocation", "/tmp/kafka/web_memory/") \
  .start()

21/12/12 02:54:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------データをいくつか送信してみます---------------------

In [5]:
# 取得したデータは順次データを取得することができます
# データを取得するにはJson_tupleを使ってデータを取り出していきます。
spark.sql("select * from web_actions LATERAL VIEW json_tuple(value,'name','action','sendtime') user as name,action,sendtime ").show(truncate=False)
result_df=spark.sql("select * from web_actions LATERAL VIEW json_tuple(value,'name','action','sendtime') user as name, action,sendtime ")

+----+---------------------------------------------------------------------------------+------------------+----------+-------------+
|key |value                                                                            |name              |action    |sendtime     |
+----+---------------------------------------------------------------------------------+------------------+----------+-------------+
|null|{"name": "yuki_1639277668791", "action": "login", "sendtime": 1639277668791}     |yuki_1639277668791|login     |1639277668791|
|null|{"name": "yuki_1639277668912", "action": "login", "sendtime": 1639277668912}     |yuki_1639277668912|login     |1639277668912|
|null|{"name": "yuki_1639277669059", "action": "login", "sendtime": 1639277669059}     |yuki_1639277669059|login     |1639277669059|
|null|{"name": "yuki_1639277669197", "action": "login", "sendtime": 1639277669197}     |yuki_1639277669197|login     |1639277669197|
|null|{"name": "yuki_1639277669364", "action": "login", "sendtime": 1

# ウィンドウ集計
データを一定期間で集計することをウィンドウ集計と呼びます。  
ストリーミングの醍醐味の一つとしてのウィンドウ集計を紹介します。  

今回は時間単位でのデータ件数の集計を行っていきましょう。
よくアプリケーションなどを使っていると、5分間の平均XXXのような値が出てきますが、中身はこれから紹介するようなコードが動いています。

Water Mark 10分の図  
Windwo 10分  
プロセスは5分  
http://mogile.web.fc2.com/spark/structured-streaming-programming-guide.html

![図1.2 DIKW](images/window.png)

In [6]:
from pyspark.sql.functions import window, col, from_json
from pyspark.sql.types import *

# データはJsonなので、Jsonの定義を行います。
schema = StructType([
    StructField('name', StringType(), True),
    StructField('action', StringType(), True),
    StructField('sendtime', StringType(), True),
])

writer = df \
.select(df.timestamp ,from_json(df.value.cast("string"), schema, {"mode" : "FAILFAST"}).alias("json")) \
.select(df.timestamp ,"json.*") \
.withWatermark("timestamp", "10 seconds") \
.groupBy(window(col("timestamp"), "10 seconds")).count() \
.writeStream \
.queryName("pyevents_per_window") \
.format("memory") \
.start()

# 少し複雑ですが、読み取ったデータをJsonとして取り出して、メモリーシンクを行っています。
# json.*は入れ子構造になっているデータを平坦化するときのお決まりです
# watermark(データが遅延できる時間)
# groupByの部分で時間単位でグループ化し10秒単位での件数を集計しています。

21/12/12 03:16:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a6f25355-bad2-4fb0-b9bd-b3323db84955. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/12 03:16:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [17]:
# データを簡単に覗いてみましょう
spark.sql("select * from pyevents_per_window").show(truncate=False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|{2021-12-12 12:16:50, 2021-12-12 12:17:00}|7    |
|{2021-12-12 12:17:00, 2021-12-12 12:17:10}|17   |
|{2021-12-12 12:17:20, 2021-12-12 12:17:30}|38   |
|{2021-12-12 12:17:10, 2021-12-12 12:17:20}|16   |
+------------------------------------------+-----+



# DIKWモデル
![図1.2 DIKW](images/DIKW.png)
DIKWモデルを紹介します

DIKWモデルでは、データのステージを「Data」「Infromation」「Knowledge」「Wisdom」として定義しています。

- Data(データ)
- Information（情報）
- Knowledge（知識）
- Wisdom（知恵）

これらの頭文字をとってDIKWモデルと呼ばれています。

ETLをすることはDataを情報や知識に変換することを指します。
情報や知識は、データから見つかるルールや関係性のことです。

## データエンリッチング
必要なデータを付与する作業のことをデータエンリッチングと呼びます。  
エンリッチング(豊かにする)という意味で、後続のデータ処理をやりやすくするためにデータに対して情報を付加することです。  
組織によって必要な情報は異なると思いますが、大半の組織でやることになるであろう

- UUID
- プロセス時間の付与

について紹介していきたいと思います。

In [19]:
import uuid
from pyspark.sql.functions import udf
from  pyspark.sql.types import StringType
uuidUdf= udf(lambda : str(uuid.uuid4()),StringType())
df_2=spark.sql("select 1 as key")
df_2.withColumn('uuid',uuidUdf())
df_2.withColumn('uuid',uuidUdf()).show(truncate=False)

+---+------------------------------------+
|key|uuid                                |
+---+------------------------------------+
|1  |15561b31-74e5-4d38-b467-0473048f8a8f|
+---+------------------------------------+



In [30]:
# dataframeにuudidを付与してみる
memory_sink2 = df \
.select(df.timestamp ,from_json(df.value.cast("string"), schema, {"mode" : "FAILFAST"}).alias("json")) \
.select(df.timestamp ,"json.*").withColumn('uuid',uuidUdf()) \
.writeStream \
.queryName("web_actions3") \
.format("memory") \
.start()

21/12/12 03:41:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7d68e0c1-f29b-4b78-9ee8-ca379f18d793. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/12 03:41:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
21/12/12 03:41:24 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
21/12/12 03:41:24 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


In [32]:
#　結果の確認
spark.sql("select * from web_actions3").show(truncate=False)

+-----------------------+------------------+------+-------------+------------------------------------+
|timestamp              |name              |action|sendtime     |uuid                                |
+-----------------------+------------------+------+-------------+------------------------------------+
|2021-12-12 12:41:24.262|yuki_1639280484260|login |1639280484260|388b81b9-cc96-4073-8b4c-72b00ef23e5c|
|2021-12-12 12:41:24.432|yuki_1639280484428|login |1639280484428|99d048aa-27b4-4416-b3e2-2c5e55c524aa|
|2021-12-12 12:41:24.566|yuki_1639280484561|login |1639280484561|6a67ef2d-4fe9-4016-96b0-b89edb2cc61d|
|2021-12-12 12:41:24.722|yuki_1639280484718|login |1639280484718|8abdb6c1-e3f3-458e-ad67-6c33f7878770|
|2021-12-12 12:41:24.873|yuki_1639280484871|login |1639280484871|10485dd3-4d32-4b2d-a7e2-ad8bd487402f|
|2021-12-12 12:41:25.024|yuki_1639280485020|login |1639280485020|94737ded-64ea-4820-a62f-30c675c33335|
|2021-12-12 12:41:25.153|yuki_1639280485148|login |1639280485148|8ab4417e

# UUIDの付与？
ストリーミングにおいては、主キー的な役割を果たします。  
大量のレコードを検索して問題のレコードを発見するのは非常に時間がかかる行為です。  

そのため、レコードを一意に特定できる値を付与するということは非常に有用です。

In [33]:
import datetime
from pyspark.sql import functions as F
spark.sql("select * from web_actions3").withColumn('spark_streaming_processed_time', F.lit(datetime.datetime.now())).show(truncate=False)

+-----------------------+------------------+------+-------------+------------------------------------+------------------------------+
|timestamp              |name              |action|sendtime     |uuid                                |spark_streaming_processed_time|
+-----------------------+------------------+------+-------------+------------------------------------+------------------------------+
|2021-12-12 12:41:24.262|yuki_1639280484260|login |1639280484260|388b81b9-cc96-4073-8b4c-72b00ef23e5c|2021-12-12 12:43:15.743575    |
|2021-12-12 12:41:24.432|yuki_1639280484428|login |1639280484428|99d048aa-27b4-4416-b3e2-2c5e55c524aa|2021-12-12 12:43:15.743575    |
|2021-12-12 12:41:24.566|yuki_1639280484561|login |1639280484561|6a67ef2d-4fe9-4016-96b0-b89edb2cc61d|2021-12-12 12:43:15.743575    |
|2021-12-12 12:41:24.722|yuki_1639280484718|login |1639280484718|8abdb6c1-e3f3-458e-ad67-6c33f7878770|2021-12-12 12:43:15.743575    |
|2021-12-12 12:41:24.873|yuki_1639280484871|login |16392804848

# いっぱい時間が出てくるけど。。？
 
データパイプラインが長くなってくると、いつどこでどんな処理をされたのか？  
というのが分かりづらくなってきます。  

どのパイプラインを通ったのか？どこの処理をいつ通ったのか？  
都度記録しておくことで、障害時のトラッキングや調査に役立ててることが可能です。

例えばsendtimeは送った時間、timestampはkafkaにたどり着いた時間、spark_streaming_processed_timeはSparkが処理した時間です。

例えばspark_streaming_processed_timeとtimestampの時間が1時間くらい離れていたらどうでしょうか？  
何かしら処理のプロセスに問題があったかもしれませんし、いきなりデータが増えて処理しきれなくなっているのかもしれません。